In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn

from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Input, LeakyReLU
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Embedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 19.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 18.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip


2023-04-18 20:43:15.346501: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import json


# convert ot pandas dataframe
cards = pd.read_json('../data/cards.json')
cards

,name,rules_text,colors,color_identity,flavour_text,type_line,power,toughness,set
0,Static Orb,"As long as CARDNAME is untapped, players can't...","[0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 1]",NaN,Artifact,None,None,7ed
1,Sensory Deprivation,Enchant creature\nEnchanted creature gets <-3/...,"[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",NaN,Enchantment — Aura,None,None,m14
2,Road of Return,Choose one —\n• Return target permanent card f...,"[0, 0, 0, 0, 1, 0]","[0, 0, 0, 0, 1, 0]",NaN,Sorcery,None,None,c19
3,Storm Crow,Flying (This creature can't be blocked except ...,"[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",NaN,Creature — Bird,1,2,9ed
4,Walking Sponge,tap: Target creature loses your choice of fly...,"[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0]",NaN,Creature — Sponge,1,1,ulg
...,...,...,...,...,...,...,...,...,...
25132,Devoted Hero,,"[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0]",NaN,Creature — Elf Soldier,1,2,s99
25133,Without Weakness,Target creature you control gains indestructib...,"[0, 0, 1, 0, 0, 0]","[0, 0, 1, 0, 0, 0]",NaN,Instant,None,None,hou
25134,Firesong and Sunspeaker,Red instant and sorcery spells you control hav...,"[1, 0, 0, 1, 0, 0]","[1, 0, 0, 1, 0, 0]",NaN,Legendary Creature — Minotaur Cleric,4,6,2x2
25135,"Samut, the Tested",+1: Up to one target creature gains double str...,"[0, 0, 0, 1, 1, 0]","[0, 0, 0, 1, 1, 0]",NaN,Legendary Planeswalker — Samut,None,None,hou


In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# get first where sum color identity is 1
# cards = cards[cards['color_identity'].apply(lambda x: len(x) == 1)]

X = []
for index, card in cards.iterrows():
    input_text = card['type_line']
    if card['rules_text'] is not None:
        input_text += '\n' + card['rules_text']
    if card['power'] is not None:
        input_text += '\n<' + card['power'] + '/' + card['toughness'] + '>'

    quoteRemoval = input_text.replace('"','')
    spaceRemoval = re.sub("\s\s+" , " ", quoteRemoval)
    stringRemoval = spaceRemoval.strip()
    specialChar = re.sub(r"[^a-zA-Z]+", ' ',stringRemoval)
    stop = set(stopwords.words('english')) #to remove stop words like is,the,that etc
    stpwords = ' '.join([i for i in specialChar.lower().split() if i not in stop])

    X.append(stpwords)


Y = cards["color_identity"]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
nr_of_targets = 5
y_train = [np.asarray(y)[0:nr_of_targets].astype('float32').ravel() for y in y_train]
y_test = [np.asarray(y)[0:nr_of_targets].astype('float32').ravel() for y in y_test]
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

2023-04-18 20:43:27.306779: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-18 20:43:27.322253: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-18 20:43:27.322576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-18 20:43:27.324153: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-18 20:43:27.324424: I tensorflow/compile

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

#Tf-idf
tf_idf = TfidfVectorizer(max_features=6000, norm='l1', ngram_range=(1, 3), stop_words='english', analyzer='word')
tfmodel = tf_idf.fit(X)

In [6]:
x_train = tfmodel.transform(x_train).toarray()
x_test = tfmodel.transform(x_test).toarray()
# max_text

In [8]:

model = keras.Sequential()
# dense layer
model.add(keras.layers.Dense(128, activation='relu', input_shape=x_train.shape[1:]))
# leaky relu
model.add(keras.layers.LeakyReLU(alpha=0.1))
# dropout
model.add(keras.layers.Dropout(0.2))
# dense layer
model.add(keras.layers.Dense(128, activation='relu'))
# leaky relu
model.add(keras.layers.LeakyReLU(alpha=0.1))
# dropout
model.add(keras.layers.Dropout(0.2))
# output layer
model.add(keras.layers.Dense(nr_of_targets, activation='sigmoid'))

from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc', f1_m, precision_m, recall_m])

In [9]:
from keras.callbacks import EarlyStopping, ModelCheckpoint



es = keras.callbacks.EarlyStopping(monitor='val_f1_m', min_delta=0, patience=3, verbose=0, mode='max', baseline=None, restore_best_weights=False)
mc = ModelCheckpoint('models/tf_idf_keras_best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
history = model.fit(x_train, y_train, batch_size=128, epochs=200, verbose=1, validation_split=0.15, callbacks=[es, mc])

Epoch 1/200


2023-04-18 20:44:11.336900: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x1fa8b670 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-18 20:44:11.336946: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-04-18 20:44:11.379628: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-18 20:44:11.798504: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-18 20:44:12.017774: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


151/151 [==============================] - 9s 37ms/step - loss: 0.5261 - acc: 0.2980 - f1_m: 0.0039 - precision_m: 0.0241 - recall_m: 0.0051 - val_loss: 0.4535 - val_acc: 0.5519 - val_f1_m: 0.0113 - val_precision_m: 0.5741 - val_recall_m: 0.0057
Epoch 2/200
151/151 [==============================] - 3s 20ms/step - loss: 0.3655 - acc: 0.5890 - f1_m: 0.4181 - precision_m: 0.8249 - recall_m: 0.3110 - val_loss: 0.3177 - val_acc: 0.6246 - val_f1_m: 0.6017 - val_precision_m: 0.7977 - val_recall_m: 0.4836
Epoch 3/200
151/151 [==============================] - 3s 20ms/step - loss: 0.2868 - acc: 0.6512 - f1_m: 0.6785 - precision_m: 0.8117 - recall_m: 0.5844 - val_loss: 0.2985 - val_acc: 0.6565 - val_f1_m: 0.6679 - val_precision_m: 0.7947 - val_recall_m: 0.5768
Epoch 4/200
151/151 [==============================] - 3s 21ms/step - loss: 0.2584 - acc: 0.6799 - f1_m: 0.7227 - precision_m: 0.8276 - recall_m: 0.6426 - val_loss: 0.2937 - val_acc: 0.6618 - val_f1_m: 0.6862 - val_precision_m: 0.7808 - v

In [10]:
# load best model
from keras.models import load_model

# model = load_model('best_model.h5')

score = model.evaluate(x_test, y_test, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

79/79 [==============================] - 1s 10ms/step - loss: 0.3571 - acc: 0.6531 - f1_m: 0.7119 - precision_m: 0.7663 - recall_m: 0.6672
Test Score: 0.35712507367134094
Test Accuracy: 0.6531423926353455


In [11]:
predictions = model.predict(x_test)

79/79 [==============================] - 0s 3ms/step


In [12]:
predictions

array([[4.1445801e-03, 8.9130591e-04, 6.8741897e-04, 1.4762735e-01,
        8.1084841e-01],
       [9.8277688e-01, 2.6632350e-02, 1.3621787e-03, 1.0046284e-03,
        5.7999842e-02],
       [3.9622203e-01, 5.4558478e-02, 1.8836770e-02, 3.2375113e-04,
        6.3465361e-04],
       ...,
       [6.8057524e-03, 4.3139422e-05, 2.4143585e-03, 1.9681004e-01,
        9.6215564e-01],
       [2.2866720e-02, 1.8920662e-02, 1.5573264e-02, 1.7926268e-01,
        6.5520388e-01],
       [9.1709770e-02, 9.8865801e-01, 3.0883173e-05, 1.5386491e-04,
        3.6124291e-04]], dtype=float32)

In [13]:
y_test

<tf.Tensor: shape=(2514, 5), dtype=float32, numpy=
array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)>

In [14]:
from preprocess.scryfall_preprocessor import ScryfallPreprocessor

sf_prepro = ScryfallPreprocessor(None)
card = sf_prepro.get_card("Bhaal, Lord of Murder")

card = sf_prepro.extract_cards([card])[0]
text = sf_prepro.stopword_preprocessing(sf_prepro.train_text(card))
print(text)
test_text = tfmodel.transform([text]).toarray()
prediction = model.predict(test_text)
colors = ['W', 'U', 'B', 'R', 'G']
for i in range(len(colors)):
    # print percentage chance of being that color
    print(colors[i] + ": " + str(prediction[0][i] * 100) + "%")
for i in range(len(colors)):
    # if over 50% chance of being that color
    if prediction[0][i] > 0.10:
        print(colors[i], end="")

Extracting cards...
legendary creature god long life total less equal half starting life total cardname indestructible whenever another nontoken creature control dies put counter target creature goad
1/1 [==============================] - 0s 21ms/step
W: 81.81838393211365%
U: 0.36973748356103897%
B: 85.59908866882324%
R: 6.194564700126648%
G: 1.0202397592365742%
WB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
